# ReAct logic test notebook

Цей ноутбук допомагає швидко перевірити підтримку structured output у LLM
та прогнати `react_loop_node` на простому прикладі.

Перед запуском переконайтеся, що vLLM/OpenAI endpoint і Graphiti доступні
та налаштовані через `config.settings` або змінні оточення.

In [1]:
from pathlib import Path
import sys
import asyncio

from pydantic import BaseModel

repo_root = Path("../graphity_lapa")
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from clients.llm_client import get_llm_client
from agent.nodes.generate_solve_response import generate_solve_response_node
from agent.state import create_initial_state

In [6]:
# Прогін react_loop_node
state = create_initial_state(
    message_uid="demo-1",
    message_text="Поясни, що таке ReAct агент простими словами",
    user_id="local_test",
)

# Приклад додаткового контексту (якщо потрібно)
state["retrieved_context"] = [
    {
        "source_msg_uid": "seed-1",
        "content": "ReAct поєднує reasoning і acting, чергуючи думки та дії.",
    }
]

result = await generate_solve_response_node(state)
print(result)

Error generating response: Connection error.
Error in generate_solve_response: Connection error.
Traceback (most recent call last):
  File "/Users/creator/Documents/Creator/PythonProject/graphity_lapa/.venv/lib/python3.12/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/Users/creator/Documents/Creator/PythonProject/graphity_lapa/.venv/lib/python3.12/site-packages/httpx/_transports/default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/creator/Documents/Creator/PythonProject/graphity_lapa/.venv/lib/python3.12/site-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "/Users/creator/Documents/Creator/PythonProject/graphity_lapa/.venv/lib/python3.12/site-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request
    response = await connection.han

{'response': 'Вибачте, сталася технічна помилка при обробці запиту.', 'references': [], 'reasoning': 'Exception: Connection error.'}
